In [1]:
from caveclient import CAVEclient
import numpy as np

import sys
sys.path.append('utils')
from FANC_utils import filter_syn_df

In [2]:
# Initialize CAVEclient
# This step requires an authentication token, see https://caveclient.readthedocs.io/en/latest/guide/authentication.html
client = CAVEclient()
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)
client.materialize.version = 840

In [19]:
timestamp = '2024-01-17T08:10:01.179472'

In [3]:
# Specify 9A IDs 
# client.materialize.version = 840
# timestamp = '2024-01-17T08:10:01.179472'
source_ids = [648518346496946148, # chief_9A
              648518346479847574, # 9A_1
              648518346479837078, # 9A_2
              648518346498002535, # 9A_3
              648518346479879156, # 9A_4
              648518346467364359, # 9A_5
              648518346486716621] # 9A_6

# Calculate 9A outputs onto hook 

In [5]:
# Specify target IDs
target_df = client.materialize.query_table('feco_axons_v0')
target_df = target_df.query('cell_type.str.contains("hook")')
target_ids = target_df.pt_root_id.to_list()

In [8]:
syn_thresh = 3
n_syn = np.zeros(len(source_ids))

for ix, i in enumerate(source_ids):
    syn_df = client.materialize.synapse_query(pre_ids=i)   
    syn_df_filt = filter_syn_df(syn_df, syn_thresh)
    syn_per_conn = syn_df_filt.groupby(['pre_pt_root_id','post_pt_root_id'])['post_pt_root_id'].count().reset_index(name="number_of_synapses").sort_values(by=["number_of_synapses"]).sort_values(by=['number_of_synapses'], ascending=False).reset_index(drop=True)
    
    # Calculate relative output   
    partner_ids = syn_per_conn.post_pt_root_id.to_list()
    for j in partner_ids:
        if j in target_ids:
            n_syn[ix] += syn_per_conn.number_of_synapses[syn_per_conn.post_pt_root_id==j].item() / sum(syn_per_conn.number_of_synapses)

In [9]:
n_syn

array([0.62783505, 0.11490683, 0.00810081, 1.        , 0.04190049,
       0.5483871 , 0.17647059])

# Calculate 9A input from sensory or descending neurons

In [29]:
# Get neuron information
neuron_info_df = client.materialize.query_table('neuron_information')

In [30]:
# Get target IDs
tag = 'sensory neuron'
#tag = 'descending'
target_ids = neuron_info_df.pt_root_id[neuron_info_df.tag==tag].tolist()

In [32]:
syn_thresh = 3
n_syn = np.zeros(len(source_ids))

for ix, i in enumerate(source_ids):
    syn_df = client.materialize.synapse_query(post_ids=i)   
    syn_df_filt = filter_syn_df(syn_df, syn_thresh)
    syn_per_conn = syn_df_filt.groupby(['pre_pt_root_id','post_pt_root_id'])['post_pt_root_id'].count().reset_index(name="number_of_synapses").sort_values(by=["number_of_synapses"]).sort_values(by=['number_of_synapses'], ascending=False).reset_index(drop=True)
    
    partner_ids = syn_per_conn.pre_pt_root_id.to_list()
    for j in partner_ids:
        if j in target_ids:
            n_syn[ix] += syn_per_conn.number_of_synapses[syn_per_conn.pre_pt_root_id==j].item() / sum(syn_per_conn.number_of_synapses)

In [33]:
n_syn

array([0.03960396, 0.32061069, 0.00099577, 0.18567639, 0.04017857,
       0.13507625, 0.04142012])

# Calculate 9A input from premotor neurons

In [34]:
# Specify target IDs
target_df = client.materialize.query_table('left_t1_local_premotor_table_v6')
target_ids = target_df.pt_root_id.to_list()

In [36]:
syn_thresh = 3
n_syn = np.zeros(len(source_ids))

for ix, i in enumerate(source_ids):
    syn_df = client.materialize.synapse_query(post_ids=i)   
    syn_df_filt = filter_syn_df(syn_df, syn_thresh)
    syn_per_conn = syn_df_filt.groupby(['pre_pt_root_id','post_pt_root_id'])['post_pt_root_id'].count().reset_index(name="number_of_synapses").sort_values(by=["number_of_synapses"]).sort_values(by=['number_of_synapses'], ascending=False).reset_index(drop=True)
    
    partner_ids = syn_per_conn.pre_pt_root_id.to_list()
    for j in partner_ids:
        if j in target_ids:
            n_syn[ix] += syn_per_conn.number_of_synapses[syn_per_conn.pre_pt_root_id==j].item() / sum(syn_per_conn.number_of_synapses)

In [37]:
n_syn

array([0.06930693, 0.12977099, 0.35997013, 0.44562334, 0.63839286,
       0.53812636, 0.70414201])